In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MultiLabelBinarizer

2024-05-02 02:39:10.848281: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-02 02:39:10.848412: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-02 02:39:11.121883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Load your data
data_path = '/kaggle/input/cafa-prot-struct-embed/Train_Structural_Embeddings_All.csv'
data = pd.read_csv(data_path)


In [ ]:
# Assuming your labels are in the 'label' column and are separated by commas
labels = data['Labels'].str.split(',')

# Use the 'sequence' column for text data
text_column = data['Sequence'].astype(str)

# Binarize labels
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(labels)


In [ ]:
# Create a set of all unique characters in the dataset
unique_chars = set(char for text in text_column for char in text)

# Create a character to integer mapping
char_to_int = {char: i + 1 for i, char in enumerate(unique_chars)}  # Start indexing from 1

# Convert all texts into sequences of integers
text_int_sequences = [[char_to_int[char] for char in text] for text in text_column]

# Pad sequences to ensure consistent input size
max_length = max(len(seq) for seq in text_int_sequences)
text_int_sequences = pad_sequences(text_int_sequences, maxlen=max_length, padding='post')

In [ ]:
# Pad sequences to ensure consistent input size
max_length = max(len(seq) for seq in text_int_sequences)
text_int_sequences = pad_sequences(text_int_sequences, maxlen=max_length, padding='post')

# Parameters for the embedding layer
num_characters = len(char_to_int) + 1  # Adding 1 because indexing starts from 1
embedding_dim = 8


In [ ]:
# Create the embedding layer
embedding_layer = tf.keras.layers.Embedding(input_dim=num_characters, output_dim=embedding_dim, input_length=max_length)

# Build the model
model = tf.keras.Sequential([
    embedding_layer,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(mlb.classes_), activation='sigmoid')  # Output layer for multi-label classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Train the model
model.fit(text_int_sequences, binary_labels, epochs=50, batch_size=32, callbacks=[early_stopping], validation_split=0.2)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
# Get embeddings
embeddings = model.layers[0](text_int_sequences).numpy()

# Save embeddings along with pdb_ids to a CSV
embedded_data = pd.DataFrame(embeddings.reshape(embeddings.shape[0], -1))  # Reshape for saving
embedded_data['pdb_id'] = data['pdb_id']  # Ensure you have a 'pdb_id' column
embedded_data.to_csv('/mnt/data/embedded_sequences.csv', index=False)